In [1]:
import keras
import tensorflow as tf

import numpy as np
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, GlobalAveragePooling2D, Input,AveragePooling2D, BatchNormalization, LeakyReLU, SpatialDropout2D

from tensorflow.keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def getSpeciesCode(x):
    part = x.split('_')
    if part[0] == 'ASETRI':
        return 0
    elif part[0] == 'EPTBOT':
        return 1
    elif part[0] == 'MYOEMA':
        return 2
    elif part[0] == 'PIPKUH':
        return 3
    elif part[0] == 'RHIMUS':
        return 4
    elif part[0] == 'RHYNAS':
        return 5
    elif part[0] == 'ROUAEG':
        return 6
    elif part[0] == 'TAPPER':
        return 7
    else:
        return 'Unknown'
    
def getSpecies(x):
    if x == 0:
        return 'A.tridens'
    elif x == 1:
        return 'E.bottae'
    elif x == 2:
        return 'M.emarginatus'
    elif x == 3:
        return 'P.kuhli'
    elif x == 4:
        return 'R.muscatellum'
    elif x == 5:
        return 'R.nasutus'
    elif x == 6:
        return 'R.aegyptius'
    elif x == 7:
        return 'T.perforatus'
    else:
        return 'Unknown'
    

In [7]:
# loading a test image
import os
from PIL import Image

X_all = []
Y_all = []

image_folder_path = '../data/SpectogramInitial'
file_paths = [f for f in os.listdir(image_folder_path)]

for file_name in file_paths:
    spectrogram = Image.open(image_folder_path + '/' + file_name)
    spectrogram = spectrogram.convert('RGB')
    spectrogram = spectrogram.resize((170, 112))  
    spectrogram = np.array(spectrogram)
    #spectrogram = np.expand_dims(spectrogram, axis=2) 
    X_all.append(spectrogram)
    Y_all.append(getSpeciesCode(file_name))
    

    
X_all = np.array(X_all)
Y_all = np.array(Y_all)




X_all = X_all.astype('float32')
X_all /= 255

Y_all = np_utils.to_categorical(Y_all, num_classes=8) 


print(X_all.shape)
print(Y_all.shape)

(3018, 112, 170, 3)
(3018, 8)


# Converting mel-spec  v1 best model to .tflite

In [8]:
import os
model_folder_path="../AI4Good/SMOTE_all_V/models"
model_file_name="mel_smote_v1_iter_1"
file_ext=".model"

tf_model_path= model_folder_path +"/"+ model_file_name + file_ext
print ("tf_model_path: " + tf_model_path)
model_to_convert=tf.keras.models.load_model(tf_model_path)
model_to_convert.summary()




tf_model_path: ../AI4Good/SMOTE_all_V/models/mel_smote_v1_iter_1.model
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 112, 170, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 85, 56)        8288      
_________________________________________________________________
batch_normalization_5 (Batch (None, 56, 85, 56)        224       
_________________________________________________________________
activation_5 (Activation)    (None, 56, 85, 56)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 43, 56)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 43, 56)        0         
______________________________________________________

In [9]:
converter=tf.lite.TFLiteConverter.from_keras_model(model_to_convert)
converted_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT] # default optimizes for both size and latency

open (model_file_name+".tflite" , "wb") .write(converted_model) # store the tflite model

818376

In [10]:

import random as random 
from random import randint
i=random.randint(1,len(Y_all) -1)
print(i)

print(Y_all.shape)
print(X_all [i].shape) 
print(X_all [i].dtype) 


2520
(8,)
(112, 170, 3)
float32


# Testing a tflite model

In [11]:
#pip install netron
# import netron
tf_lite_model_file=model_file_name+".tflite"
# netron.start(tf_lite_model_file, port=8081)

In [20]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.

converted_model_path=tf_lite_model_file
interpreter = tf.lite.Interpreter(model_path=converted_model_path)
interpreter.allocate_tensors()  #- allocates tensors to the model 

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = X_all [i].reshape (1,112,170,3)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()  # runs the test 

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Expected output: ",getSpecies(np.argmax(output_data)))
print ("Actual Output: ",getSpecies(np.argmax(Y_all [i])))

Expected output:  M.emarginatus
Actual Output:  M.emarginatus


In [ ]:
# .model file = 580 kB
# .tflite file= 818 kB

## Appendix- what each interpreter function does or looks like

In [11]:
input_details = interpreter.get_input_details()
print (input_details) # stores the details in key value format of the input layer at the top

[{'name': 'input_13', 'index': 0, 'shape': array([  1, 112, 170,   3]), 'shape_signature': array([ -1, 112, 170,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [13]:
output_details = interpreter.get_output_details()
print (output_details) # stores the details in key value format of the output layer at the the end of the model

[{'name': 'Identity', 'index': 25, 'shape': array([1, 8]), 'shape_signature': array([-1,  8]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [19]:
output_details[0]['index']

25

In [21]:
print (interpreter.allocate_tensors())

None
